## Introduction

In this project, Beautiful Soup Python library was used to scrap data from Amazon website. Results of search using “multivitamin for kids”  keyword was scrapped from. The result was converted to a Pandas dataframe using pandas library and saved as csv file.

Because Amazon website is a dynamic site constructed with Javascript, tools like Selenium or Splash is needed. In this project [Splash](https://splash.readthedocs.io/en/stable/index.html) was used.

## Exploring the first page

In [1]:
import requests
import re
from bs4 import BeautifulSoup as bs
import pandas as pd

In [2]:
# make sure that splash is runing through Docker

serach_key = 'multivitamin for kids' ## search keyword used for product.

serach_key = serach_key.replace(' ', '+')


url = f'https://www.amazon.com/s?k={serach_key}&page={1}'

r = requests.get('http://localhost:8050/render.html',params={'url':url, 'wait':4})# Splash used torender the pages

soup = bs(r.text, 'html.parser')

In [3]:
print(r) # Ensuring the code executed successfully

<Response [200]>


In [4]:
# Printing first item in the data
products= soup.findAll('div', {'data-component-type' : re.compile("s-search-result")})
products[0]

<div class="sg-col-4-of-24 sg-col-4-of-12 s-result-item s-asin sg-col-4-of-16 AdHolder sg-col s-widget-spacing-small sg-col-4-of-20" data-asin="B0B6WQ7NK8" data-cel-widget="search_result_2" data-component-id="15" data-component-type="s-search-result" data-index="2" data-uuid="bc9aa370-5cec-499d-bbd5-94e1cbd1f9d2"><div class="sg-col-inner"><div cel_widget_id="MAIN-SEARCH_RESULTS-2" class="s-widget-container s-spacing-small s-widget-container-height-small celwidget slot=MAIN template=SEARCH_RESULTS widgetId=search-results_1" data-cel-widget="MAIN-SEARCH_RESULTS-2" data-csa-c-id="rejuv6-tcwk97-j96ozs-ogzajr" data-csa-c-item-id="amzn1.asin.1.B0B6WQ7NK8" data-csa-c-pos="1" data-csa-c-type="item" data-csa-op-log-render="">
<div class="rush-component s-expand-height" data-component-props='{"percentageShownToFire":"50","batchable":true,"requiredElementSelector":".s-image:visible","url":"https://unagi-na.amazon.com/1/events/com.amazon.eel.SponsoredProductsEventTracking.prod?qualifier=1675456041

In [5]:
# Number of products in first page.
len(products)

60

## Pagination 
Collecting all search result data from differen pages.

In [6]:

serach_key = 'multivitamin for kids'

serach_key = serach_key.replace(' ', '+').strip()

results_list = []



for x in range(1,10):

    url = f'https://www.amazon.com/s?k=f{serach_key}&page={x}'

    print(url)
    
    
    r = requests.get('http://localhost:8050/render.html',params={'url':url, 'wait':4})

    soup = bs(r.text, 'html.parser')
    
    links_href = soup.find('span', class_= re.compile('s-pagination-strip'))
    
    if not links_href.find('span', class_ =  re.compile('s-pagination-item s-pagination-next s-pagination-disabled')):
        pass
    else:
        break
    
    for item in soup.findAll('div', {'data-component-type' : re.compile("s-search-result")}):
        
        description = item.find('span', class_= re.compile('a-size-base-plus a-color-base a-text-normal'))

        try:
            number_rates = item.find('span', class_= re.compile('a-size-base.s-underline-text')).text
        except: number_rates = 0

        try:
            rating = item.find('span', class_= re.compile('a-icon-alt')).string.split()[0]
        except: 
            rating = 0

        try:
            price = item.find('span', class_='a-offscreen').text.replace('$','')
        except:
             price = 'none'

        link= item.find('a').attrs['href']
        url = 'https://www.amazon.com' + link

        product = {
            'description': description,
            'number_rates': number_rates,
            'rating' : rating,
            'price' : price,
            'url' : url     
        }

        results_list.append(product)

    


https://www.amazon.com/s?k=fmultivitamin+for+kids&page=1
https://www.amazon.com/s?k=fmultivitamin+for+kids&page=2
https://www.amazon.com/s?k=fmultivitamin+for+kids&page=3
https://www.amazon.com/s?k=fmultivitamin+for+kids&page=4
https://www.amazon.com/s?k=fmultivitamin+for+kids&page=5
https://www.amazon.com/s?k=fmultivitamin+for+kids&page=6
https://www.amazon.com/s?k=fmultivitamin+for+kids&page=7


In [7]:
## Total number of products
print(len(results_list))

360


In [8]:
## to make pandas dataframe
df = pd.DataFrame(results_list) 
df.head()

,description,number_rates,rating,price,url
0,"[Amazon Brand - Solimo Kids' Multivitamin, 380...","(1,300)",4.8,11.88,https://www.amazon.com/gp/slredirect/picassoRe...
1,"[Centrum Kids Multivitamin Gummies, Tropical P...","(1,240)",4.7,14.28,https://www.amazon.com/gp/slredirect/picassoRe...
2,[Renzo's Picky Eater Kids Multivitamin - Vegan...,"(4,030)",4.3,25.99,https://www.amazon.com/gp/slredirect/picassoRe...
3,[Flintstones Kids Elderberry Gummies with Immu...,(807),4.7,13.43,https://www.amazon.com/gp/slredirect/picassoRe...
4,"[OLLY Kids Multivitamin + Probiotic Gummy, Dig...","(13,553)",4.7,11.99,https://www.amazon.com/gp/bestsellers/hpc/3765...


## Saving dataset 

In [9]:
## to save data in a csv file.
df.to_csv('Multivitamines_kids.csv')